<a href="https://colab.research.google.com/github/MachineNeyarning/ClassifiersCommittee/blob/main/ClassifiersCommittee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comitê de Classificadores | Projeto N1

**GitHub: https://github.com/MachineNeyarning/ClassifiersCommittee**

---

## Grupo:
- Gustavo Henrique Martins
- Ícaro Botelho
- Maria Clara Seixa Scheffel
- Maruan Biasi El Achkar
- Ricardo Falcão Schlieper

### Dataset escolhido: Rain in Australia
- Detalhamento de colunas em: https://github.com/MachineNeyarning/ClassifiersCommittee/tree/main/dataset
- Fonte: https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package
- Target: RainTomorrow

### Algoritmos escolhidos:
- K-Nearest Neighbors (KNN)
- Decision Tree Classifier (Tree-Based)
- Random Forest Classifier (Tree-Based)
- LightGBM Classifier (Tree-Based)
- CatBoost Classifier (Tree-Based)
- Gaussian Naive Bayes (Naive Bayes)
- Support Vector Classification (SVC)
- Multi-Layer Perceptron (MLPClassifier)
- AdaBoost Classifier (Ensemble Methods)
- Quadratic Discriminant Analysis (QDA)



---
# Preparar Dataset

In [ ]:
# Baixar Dataset

import kagglehub
import os

path = kagglehub.dataset_download("jsphyg/weather-dataset-rattle-package")
csv_path = os.path.join(path, 'weatherAUS.csv')


Using Colab cache for faster access to the 'weather-dataset-rattle-package' dataset.


In [ ]:
# Criar Dataframe Pandas

import pandas as pd

df = pd.read_csv(csv_path, sep=',', engine='python', on_bad_lines='skip')
df.head(3)

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No


In [ ]:
# Describe Dataframe Data

df.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
count,143975.000000,144199.000000,142199.000000,82670.000000,75625.000000,135197.000000,143693.000000,142398.000000,142806.000000,140953.000000,130395.00000,130432.000000,89572.000000,86102.000000,143693.000000,141851.00000
mean,12.194034,23.221348,2.360918,5.468232,7.611178,40.035230,14.043426,18.662657,68.880831,51.539116,1017.64994,1015.255889,4.447461,4.509930,16.990631,21.68339
std,6.398495,7.119049,8.478060,4.193704,3.785483,13.607062,8.915375,8.809800,19.029164,20.795902,7.10653,7.037414,2.887159,2.720357,6.488753,6.93665
min,-8.500000,-4.800000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,980.50000,977.100000,0.000000,0.000000,-7.200000,-5.40000
25%,7.600000,17.900000,0.000000,2.600000,4.800000,31.000000,7.000000,13.000000,57.000000,37.000000,1012.90000,1010.400000,1.000000,2.000000,12.300000,16.60000
50%,12.000000,22.600000,0.000000,4.800000,8.400000,39.000000,13.000000,19.000000,70.000000,52.000000,1017.60000,1015.200000,5.000000,5.000000,16.700000,21.10000
75%,16.900000,28.200000,0.800000,7.400000,10.600000,48.000000,19.000000,24.000000,83.000000,66.000000,1022.40000,1020.000000,7.000000,7.000000,21.600000,26.40000
max,33.900000,48.100000,371.000000,145.000000,14.500000,135.000000,130.000000,87.000000,100.000000,100.000000,1041.00000,1039.600000,9.000000,9.000000,40.200000,46.70000


In [ ]:
# Verificar falta de dados

missing_data = df.isnull().sum()
display(missing_data)

,0
Date,0
Location,0
MinTemp,1485
MaxTemp,1261
Rainfall,3261
Evaporation,62790
Sunshine,69835
WindGustDir,10326
WindGustSpeed,10263
WindDir9am,10566


In [ ]:
# Remover linhas com dados faltando

df_cleaned = df.dropna()
display(df_cleaned.head())
print(f"Original: {df.shape}")
print(f"Limpo: {df_cleaned.shape}")

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
6049,2009-01-01,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,ENE,...,20.0,13.0,1006.3,1004.4,2.0,5.0,26.6,33.4,No,No
6050,2009-01-02,Cobar,18.4,28.9,0.0,14.8,13.0,S,37.0,SSE,...,30.0,8.0,1012.9,1012.1,1.0,1.0,20.3,27.0,No,No
6052,2009-01-04,Cobar,19.4,37.6,0.0,10.8,10.6,NNE,46.0,NNE,...,42.0,22.0,1012.3,1009.2,1.0,6.0,28.7,34.9,No,No
6053,2009-01-05,Cobar,21.9,38.4,0.0,11.4,12.2,WNW,31.0,WNW,...,37.0,22.0,1012.7,1009.1,1.0,5.0,29.1,35.6,No,No
6054,2009-01-06,Cobar,24.2,41.0,0.0,11.2,8.4,WNW,35.0,NW,...,19.0,15.0,1010.7,1007.4,1.0,6.0,33.6,37.6,No,No


Original: (145460, 23)
Limpo: (56420, 23)


---
# 12 Algoritimos:

---
## 1) K-Nearest Neighbors (KNN) |

In [ ]:
# Copia do dataframe
df_knn = df_cleaned

###Importar as Bibliotecas

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score


# Excluir a coluna 'Date' se não for relevante
df_knn = df_knn.drop(columns=['Date'])

# Converter variáveis categóricas em variáveis dummy (one-hot encoding)
df_knn = pd.get_dummies(df_knn, drop_first=True)

# Separar variáveis independentes (X) e dependentes (y)
X = df_knn.drop(columns=['PlayTennis_Yes'])  # Substitua 'PlayTennis_Yes' pelo nome da coluna alvo
y = df_knn['PlayTennis_Yes']


In [ ]:
# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Inicializar e treinar o modelo KNN
knn = KNeighborsClassifier(n_neighbors=5) # Você pode ajustar o número de vizinhos
knn.fit(X_train_scaled, y_train)

# Fazer previsões
y_pred = knn.predict(X_test_scaled)

# Avaliar o modelo
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"KNN Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:")
print(report)

---
## 2) Decision Tree Classifier (Tree-Based) | Gustavo

In [ ]:
# Copia do dataframe
df_dtc = df_cleaned

In [ ]:
# ========================
# Decision Tree Classifier
# ========================
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

# features (X) e target (y)
X = df_dtc.drop(["RainTomorrow", 'Date', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday'], axis=1)  # variável alvo = "RainTomorrow"
y = df_dtc["RainTomorrow"]

# converter TARGET para numerico
le = LabelEncoder()
y = le.fit_transform(y)

# treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# modelo e prediçoes
dtc = DecisionTreeClassifier(random_state=42)
dtc.fit(X_train, y_train)
y_pred_dtc = dtc.predict(X_test)

# métricas
print("Decision Tree")
print("Acurácia:", accuracy_score(y_test, y_pred_dtc))
print(confusion_matrix(y_test, y_pred_dtc))
print(classification_report(y_test, y_pred_dtc))

# ROC Curve
y_prob_dtc = dtc.predict_proba(X_test)[:,1]
fpr, tpr, _ = roc_curve(y_test, y_prob_dtc)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label=f'Decision Tree (AUC = {roc_auc:.2f})')

# Keep the plot code from the Random Forest cell to display all ROC curves together
# plt.plot([0,1], [0,1], "k--")
# plt.xlabel("False Positive Rate")
# plt.ylabel("True Positive Rate")
# plt.title("ROC Curve - Tree Based Models")
# plt.legend()
# plt.show()

---
## 3) Random Forest Classifier (Tree-Based) | Gustavo

In [ ]:
# Copia do dataframe
df_rfc = df_cleaned

In [ ]:
# ========================
# Random Forest Classifier
# ========================
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

# features (X) e target (y)
X = df_rfc.drop(["RainTomorrow", 'Date', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday'], axis=1)
y = df_rfc["RainTomorrow"]

# Convert TARGET to numeric
le = LabelEncoder()
y = le.fit_transform(y)

# treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# modelo e prediçoes
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(X_train, y_train)

y_pred_rfc = rfc.predict(X_test)

# métricas
print("Random Forest")
print("Acurácia:", accuracy_score(y_test, y_pred_rfc))
print(confusion_matrix(y_test, y_pred_rfc))
print(classification_report(y_test, y_pred_rfc))

# ROC Curve
y_prob_rfc = rfc.predict_proba(X_test)[:,1]
fpr, tpr, _ = roc_curve(y_test, y_prob_rfc)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label=f'Random Forest (AUC = {roc_auc:.2f})')

# gráfico ROC
plt.plot([0,1], [0,1], "k--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - Tree Based Models")
plt.legend()
plt.show()

---
## 4) XGBoost (Tree-Based) | Maruan

In [ ]:
# Copia do dataframe
df_xgb = df_cleaned

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [ ]:
# Remover tabelas nao numericas e o target
X_xgb = df_xgb.drop(['Date', 'Location', 'RainToday', 'RainTomorrow', 'WindGustDir', 'WindDir9am', 'WindDir3pm'], axis=1)
y_xgb = df_xgb['RainTomorrow']

# Converter TARGET para numerico
le = LabelEncoder()
y_xgb = le.fit_transform(y_xgb)

In [ ]:
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(X_xgb, y_xgb, test_size=0.2, random_state=42, stratify=y_xgb)

In [ ]:
# XGBoost model

# resultado com parametros padrao: 85.69%
xgboost = xgb.XGBClassifier(random_state=42, eval_metric='logloss')

xgboost.fit(X_train_xgb, y_train_xgb)

In [ ]:
# Avaliacao

y_pred_xgb = xgboost.predict(X_test_xgb)
accuracy_xgb = accuracy_score(y_test_xgb, y_pred_xgb)

print(f"XGBoost Accuracy: {accuracy_xgb * 100:.2f}%")

---
## 5) LightGBM Classifier (Tree-Based) | Maruan

In [ ]:
# Copia do dataframe
df_lgbm = df_cleaned

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [ ]:
# Remover tabelas nao numericas e o target
X_lgbm = df_lgbm.drop(['Date', 'Location', 'RainToday', 'RainTomorrow', 'WindGustDir', 'WindDir9am', 'WindDir3pm'], axis=1)
y_lgbm = df_lgbm['RainTomorrow']

# Converter TARGET para numerico
le = LabelEncoder()
y_lgbm = le.fit_transform(y_lgbm)

In [ ]:
X_train_lgbm, X_test_lgbm, y_train_lgbm, y_test_lgbm = train_test_split(X_lgbm, y_lgbm, test_size=0.2, random_state=42, stratify=y_lgbm)

In [ ]:
# LightGBM model

# resultado com parametros padrao: 85.90%
# melhor resultado com parametros personalizados: 86.46%
lgbm = lgb.LGBMClassifier(
    random_state=42,
    n_estimators=52783,
    learning_rate=0.055,
    num_leaves=80,
    max_depth=-1,
    max_bin=716,
    min_data_in_leaf=60,
    min_sum_hessian_in_leaf=5.0,
    min_gain_to_split=0.01,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=1,
    lambda_l1=0.1,
    lambda_l2=2.0,
    # extra_trees=True,
    # class_weight="balanced",
    n_jobs=-1,
    verbose=-1 # para nao encher de log no console senao o colab fica com 500mb
)

lgbm.fit(
    X_train_lgbm, y_train_lgbm,
    eval_set=[(X_test_lgbm, y_test_lgbm)],
    eval_metric=["auc", "binary_logloss"],
    callbacks=[lgb.early_stopping(100)]
)

In [ ]:
# Avaliacao
y_pred_lgbm = lgbm.predict(X_test_lgbm)
accuracy_lgbm = accuracy_score(y_test_lgbm, y_pred_lgbm)

print(f"LightGBM Accuracy: {accuracy_lgbm * 100:.2f}%")

---
## 6) CatBoost Classifier (Tree-Based) | Maruan

In [ ]:
# Copia do dataframe
df_cb = df_cleaned

In [ ]:
!pip install catboost

import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [ ]:
# Remover tabelas nao numericas e o target
X_cb = df_cb.drop(['Date', 'Location', 'RainToday', 'RainTomorrow', 'WindGustDir', 'WindDir9am', 'WindDir3pm'], axis=1)
y_cb = df_cb['RainTomorrow']

# Converter TARGET para numerico
le = LabelEncoder()
y_cb = le.fit_transform(y_cb)

In [ ]:
X_train_cb, X_test_cb, y_train_cb, y_test_cb = train_test_split(X_cb, y_cb, test_size=0.2, random_state=42, stratify=y_cb)

In [ ]:
# CatBoost Model

# resultado com parametros padrao: 86.46%
# melhor resultado com parametros personalizados: 86.37%

catboost = cb.CatBoostClassifier(
    random_state=40,
    # iterations=5000,
    # learning_rate=0.1,
    # depth=6,
    # l2_leaf_reg=3,
    # model_size_reg=None,
    # colsample_bylevel=0.8,
    # loss_function='Logloss',
    # eval_metric='Accuracy',
    # leaf_estimation_iterations=10,
    # bootstrap_type='Bayesian',
    # bagging_temperature=0.3,
    # random_strength=0.2,
    # auto_class_weights="Balanced",
    # od_type='Iter',
    # od_wait=50,
    verbose=0,
)

catboost.fit(X_train_cb, y_train_cb)

In [ ]:
# Avaliacao
y_pred_cb = catboost.predict(X_test_cb)
accuracy_cb = accuracy_score(y_test_cb, y_pred_cb)

print(f"CatBoost Accuracy: {accuracy_cb * 100:.2f}%")

---
## 7) Perceptron | Maruan

In [ ]:
# Copia do dataframe
df_perceptron = df_cleaned

In [ ]:
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score

In [ ]:
# Remover tabelas nao numericas e o target
X_perceptron = df_perceptron.drop(['Date', 'Location', 'RainToday', 'RainTomorrow', 'WindGustDir', 'WindDir9am', 'WindDir3pm'], axis=1)
y_perceptron = df_perceptron['RainTomorrow']

# Converter TARGET para numerico
le = LabelEncoder()
y_perceptron = le.fit_transform(y_perceptron)

In [ ]:
X_train_perceptron, X_test_perceptron, y_train_perceptron, y_test_perceptron = train_test_split(X_perceptron, y_perceptron, test_size=0.2, random_state=42, stratify=y_perceptron)

In [ ]:
# escalar
scaler = StandardScaler()
X_train_scaled_perceptron = scaler.fit_transform(X_train_perceptron)
X_test_scaled_perceptron = scaler.transform(X_test_perceptron)

In [ ]:
# Perceptron model

perceptron = Perceptron(random_state=43)

perceptron.fit(X_train_scaled_perceptron, y_train_perceptron)

In [ ]:
# Avaliacao

y_pred_perceptron = perceptron.predict(X_test_scaled_perceptron)

accuracy_perceptron = accuracy_score(y_test_perceptron, y_pred_perceptron)

print(f"Perceptron Accuracy: {accuracy_perceptron * 100:.2f}%")

---
## 8) Gaussian Naive Bayes (Naive Bayes) |

In [ ]:
# Copia do dataframe
df_gnb = df_cleaned

---
## 9) Support Vector Classification (SVC) |

In [ ]:
# Copia do dataframe
df_svc = df_cleaned

---
## 10) Multi-Layer Perceptron (MLPClassifier) |

In [ ]:
# Copia do dataframe
df_mlp = df_cleaned

---
## 11) AdaBoost Classifier (Ensemble Methods) |

In [ ]:
# Copia do dataframe
df_ada = df_cleaned

---
## 12) Quadratic Discriminant Analysis (QDA) |

In [ ]:
# Copia do dataframe
df_qda = df_cleaned

---
# Comite Classificador:

In [ ]:
# Copia do dataframe
df_comittee = df_cleaned

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Remover tabelas nao numericas e o target
X_committee = df_comittee.drop(['Date', 'Location', 'RainToday', 'RainTomorrow', 'WindGustDir', 'WindDir9am', 'WindDir3pm'], axis=1)
y_committee = df_comittee['RainTomorrow']

# Converter TARGET para numerico
le = LabelEncoder()
y_committee = le.fit_transform(y_committee)

In [ ]:
# mesma coisa dos classifiers mas para o comite
X_train_committee, X_test_committee, y_train_committee, y_test_committee = train_test_split(X_committee, y_committee, test_size=0.2, random_state=42, stratify=y_committee)

In [ ]:
# QUAIS CLASSIFICADORES VAO SER USADOS NO COMITE - QUANDO TERMINAREM UM ALGORITIMO NOVO, COLOCA ELE AQUI !!!!!!!!!!!!!
estimators = [('lgbm', lgbm), ('catboost', catboost), ('xgboost', xgboost), ('perceptron', perceptron)]

In [ ]:
# Classifiers Comittee

# Tipos de voting
# 'hard' = maioria
# 'soft' usa pesos, weights=[]
weights_acc = [accuracy_lgbm, accuracy_cb] # pesos sendo a acurracy de cada modelo

committee = VotingClassifier(estimators=estimators, voting='hard')


committee.fit(X_train_committee, y_train_committee)

In [ ]:
# Avaliacao

from sklearn.metrics import f1_score

# RESULTADOS COM APENAS LightGBM e CatBoost (maruan)
# Acurracy com HARD VOTING: 85.97%
# Acurracy com SOFT e accuracy dos algoritimos como weights: 85.86%
# tentei fazer um grid search pelos melhores pesos mas rodou por 6 horas e crashou
# cada vez que eu boto um algoritimo novo fica pior KKKKKKKKKKKKKKKKKKKK

accuracy_committee = committee.score(X_test_committee, y_test_committee)
y_pred_committee = committee.predict(X_test_committee)
accuracyf1_committee = f1_score(y_test_committee, y_pred_committee)


print(f"Classifier Committee Accuracy: {accuracy_committee * 100:.2f}%")
print(f"Classifier Committee F1-score: {accuracyf1_committee * 100:.2f}%")

print('------------------------------------------')
algorithm_names = [name for name, _ in estimators]
print(f"Algoritmos do Comite: {algorithm_names}")


---
#### sugestao de outros algoritimos:


*   Logistic Regression
*   SGDClassifier
*   Passive Aggressive Classifier (nao sei se funcionaria aqui)
*   Extra Trees Classifier (tipo um random forest ao quadrado)
*   GradientBoostingClassifier
*   HistGradientBoostingClassifier (uma versao customizada do XGBoost/LightGBM)
*   Linear Discriminant Analysis
*   LogitBoost
*   Bagged Perceptrons
*   AdaBoost com perceptrons dentro

